# Сборный проект №2

**Описание проекта**

Изучение воронки продаж и исследование результатов A/A/B-эксперимента.

**План работы**

1. Открытие файла с данными и изучиние общей информации
2. Подготовка данных
3. Изучение и проверка данных
- Количество событий
- Количество пользователей
- Среднее количествово событий на пользователя
- Распределение данных во времени
- Количество отброшенных данных
- Проверка пользователей в группах
4. Изучение воронки событий
- События по частоте
- События по числу пользователей
- Предположение о порядке событий
- Анализ воронки
5. Изучение результатов эксперимента
- Количество пользователей в каждой экспериментальной группе
- A/A-тест
- A/B-тесты
6. Выводы

## Открытие файла с данными и изучиние общей информации

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import math
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from scipy import stats as st
from plotly import graph_objects as go
import plotly.express as px

In [2]:
logs = pd.read_csv('https://code.s3.yandex.net/datasets/logs_exp.csv', sep='\t')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/logs_exp.csv'

In [ ]:
pd.options.display.max_colwidth = 80
logs.head(10)

In [ ]:
logs.info()

## Подготовка данных

### Предобработка данных

**Переименуем столбцы.**

In [ ]:
logs.rename(columns={'EventName' : 'event', 
                     'DeviceIDHash' : 'user', 
                     'EventTimestamp' : 'timestamp', 
                     'ExpId' : 'group'}, 
            inplace=True)

In [ ]:
logs.info()

**Добавляем столбец с датой и временем и столбец с датой.**

In [ ]:
logs['datetime'] = pd.to_datetime(logs['timestamp'], unit='s')

In [ ]:
logs['date'] = logs['datetime'].dt.date

In [ ]:
logs['date'] = logs['date'].astype('datetime64')

**Переименуем названия групп для удобства.**

In [ ]:
def new_group(g):
    if g == 246:
        return 'A246'
    elif g == 247:
        return 'A247'
    elif g == 248:
        return 'B'
    else:
        return 'Unknown'

logs['group'] = logs['group'].apply(new_group)

In [ ]:
logs['group'] = logs['group'].astype('str')

In [ ]:
logs['group'].value_counts()

In [ ]:
logs.head(10)

In [ ]:
logs.info()

**Вывод**

Пропусков в данных нет. Переименовали названия столбцов. Добавили столбцы с датой и временем. Переименовали названия групп.

### Работа с дубликатами

In [ ]:
logs['event'].unique()

In [ ]:
d = logs.duplicated(['event', 'user', 'datetime', 'group']).sum()
p = d / logs.shape[0]
print(f'Количество дубликатов: {d} ({p:.3%} от общего количества строк)')

**Дубликаты можно удалить.**

In [ ]:
logs = logs.drop_duplicates()
logs = logs.reset_index(drop=True)

In [ ]:
logs.head(10)

In [ ]:
logs.groupby('user').agg({'group' : 'nunique'}).query('group > 1')

**Вывод**

Удалили 413 дубликатов. Пересечений пользователей по группам нет.

## Изучение и проверка данных

### Количество событий

In [ ]:
events = logs['event'].count()
print('Количество событий в логе:', events)

In [ ]:
logs['event'].value_counts()

**Вывод**

В логе всего 243713 события.

### Количество пользователей

In [ ]:
users = logs['user'].nunique()
print('Количество событий в логе:', users)

**Вывод**

В логе 7551 уникальный пользователь.

### Среднее количествово событий на пользователя

In [ ]:
print("Среднее количество событий на пользователя: ", (events / users).round(1))

**Проверим выбросы.**

In [ ]:
event_per_user = logs.groupby('user')['event'].count()
event_per_user.describe()

In [ ]:
plt.figure(figsize=(16,8))
plt.hist(event_per_user, bins=100, range=(200,2400)) 
plt.xlabel('Количество пользователей')
plt.grid()
plt.ylabel('Количество событий')
plt.title('Количество событий на пользователя');

**Вывод**

"Длинный хвост" указывает на наличие "выбросов" в исходных данных.

### Распределение данных во времени

In [ ]:
print(f"Период данных которым мы располагаем: с {logs['datetime'].min()} по {logs['datetime'].max()}")
print(f"Размер периода: {logs['datetime'].max() - logs['datetime'].min()}")

In [ ]:
event_per_day = logs.groupby('date')['event'].count()

In [ ]:
event_per_day

In [ ]:
plt.figure(figsize=(16,8))
plt.hist(logs['datetime'], bins=14*24) 
plt.xlabel('Время')
plt.grid()
plt.ylabel('Количество событий')
plt.title('Распределение событий по времени');

In [ ]:
plt.figure(figsize=(16,8))
plt.hist(logs[logs['date'] > '2019-07-31']['datetime'], bins=7*24) 
plt.xlabel('Время')
plt.grid()
plt.ylabel('Количество событий')
plt.title('Распределение событий по времени');

In [ ]:
plt.figure(figsize=(16,8))
plt.hist(logs.loc[(logs['datetime'] >= '2019-07-31 18:00') & (logs['datetime'] <= '2019-07-31 23:00')]['datetime'], bins=5*10) 
plt.xlabel('Время')
plt.grid()
plt.ylabel('Количество событий')
plt.title('Распределение событий по времени');

In [ ]:
logs['dt10m'] = logs['datetime'].dt.round('10T')
dt10m_event = logs.groupby('dt10m')['event'].count().reset_index()
dt10m_event = dt10m_event[(dt10m_event['dt10m'] >= '2019-07-31 00:00') & (dt10m_event['dt10m'] <= '2019-08-01 00:00')]

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(dt10m_event['dt10m'], dt10m_event['event'])
plt.xlabel('Время')
plt.grid()
plt.ylabel('Количество событий')
plt.title('Распределение событий по времени');

**Вывод**

Судя по гистограммам можно сказать, что реально мы располагаем данными только за первые 7 дней августа. Данные в июле характеризуются малой событийностью. Скорее всего это было какое-то тестирование, а сбор данных начался уже со второй недели. <br>
Также отлично видно, что распределение событий в августе идет циклично - с утра количествово событий нарастает, днем держится на максимумах, а вечером падает на минимум. <br>
Из последних двух графиков можно выделить границу, где появляется существенное различие в распределении - это около 21:00 31 июля. Всё что раньше этого времени отбросим и будем проводить тестирование с данными примерно за 7 дней.

### Количество отброшенных данных

In [ ]:
logs = logs.loc[logs['datetime'] > '2019-07-31 21:00:00'].reset_index(drop=True)
logs.sort_values(by='datetime')

In [ ]:
new_users = logs['user'].nunique()
print('Было пользователей:',users)
print('Стало пользователей:',new_users)
print('Количество пользователей уменьшилось на', users - new_users)
print('Потеря:', round((users-new_users)/users*100,2),'%')

In [ ]:
new_events = logs['event'].count()
print('Было пользователей:',events)
print('Стало пользователей:',new_events)
print('Количество пользователей уменьшилось на', events - new_events)
print('Потеря:', round((events-new_events)/events*100,2),'%')

**Вывод**

Отбросив неделю июля, мы потеряли менее процента исходных данных, что тоже вполне допустимо.

### Проверка пользователей в группах

In [ ]:
logs['group'].value_counts()

In [ ]:
logs.groupby('group')['user'].nunique()

**Вывод**

В каждой группе примерно по 2500 пользователей.

## Изучение воронки событий

### События по частоте

In [ ]:
print('События в порядке убывания частоты:')
logs['event'].value_counts()

**Возможная расшифровка событий:**
* MainScreenAppear - Появление "Главного экрана"
* OffersScreenAppear - Появление экрана "Предложения"
* CartScreenAppear - Появление экрана "Корзина"
* PaymentScreenSuccessful - Появление экрана "Платеж успешен"
* Tutorial - Руководство пользователя

In [ ]:
plt.figure(figsize=(16,10))
plt.title('Частота событий');
logs['event'].value_counts().plot(kind='pie', autopct='%0.1f%%');

**Вывод**

Почти половина событий это - появление "Главного экрана", вторая половина это другие 3 события с примерно равными частотами. <br> "Руководство пользователя" очень редкое событие.

### События по числу пользователей

In [ ]:
events_users = logs.groupby('event').agg({'event':'count', 'user':'nunique'}).sort_values(by='user', ascending=False)
events_users.columns = ['count_events', 'users']
events_users = events_users.reset_index()
events_users['users_part_1event'] = (events_users['users'] / new_users * 100).round(1) # доля хотя бы раз совершивших событие

In [ ]:
events_users

In [ ]:
plt.figure(figsize=(16,8))
plt.bar(events_users['event'], events_users['users'])
plt.title('Частота событий')
plt.xlabel('Событие')
plt.grid()
plt.ylabel('Количество пользователей');

In [ ]:
plt.figure(figsize=(16,8))
plt.bar(events_users['event'], events_users['users_part_1event'])
plt.title('Доля пользователей, которые хоть раз совершали событие')
plt.xlabel('Событие')
plt.grid()
plt.ylabel('Доля пользователей');

### Предположение о порядке событий

Проверим во всех ли группах одинаковая зависимость событие по количеству событий.

In [ ]:
new_users_A246 = logs[logs['group']=='A246']['user'].nunique()
events_users_A246 = logs[logs['group']=='A246'].groupby('event').agg({'event':'count', 'user':'nunique'}).sort_values(by='user', ascending=False)
events_users_A246.columns = ['count_events', 'users']
events_users_A246 = events_users_A246.reset_index()
events_users_A246['users_part_1event'] = (events_users_A246['users'] / new_users_A246 * 100).round(1)
events_users_A246

In [ ]:
plt.figure(figsize=(16,8))
plt.bar(events_users_A246['event'], events_users_A246['users_part_1event'])
plt.title('Доля пользователей, которые хоть раз совершали событие (группа A246)')
plt.xlabel('Событие')
plt.grid()
plt.ylabel('Доля пользователей в группе');

In [ ]:
new_users_A247 = logs[logs['group']=='A247']['user'].nunique()
events_users_A247 = logs[logs['group']=='A247'].groupby('event').agg({'event':'count', 'user':'nunique'}).sort_values(by='user', ascending=False)
events_users_A247.columns = ['count_events', 'users']
events_users_A247 = events_users_A247.reset_index()
events_users_A247['users_part_1event'] = (events_users_A247['users'] / new_users_A247 * 100).round(1)
events_users_A247

In [ ]:
plt.figure(figsize=(16,8))
plt.bar(events_users_A247['event'], events_users_A247['users_part_1event'])
plt.title('Доля пользователей, которые хоть раз совершали событие (группа A247)')
plt.xlabel('Событие')
plt.grid()
plt.ylabel('Доля пользователей в группе');

In [ ]:
new_users_B = logs[logs['group']=='B']['user'].nunique()
events_users_B = logs[logs['group']=='B'].groupby('event').agg({'event':'count', 'user':'nunique'}).sort_values(by='user', ascending=False)
events_users_B.columns = ['count_events', 'users']
events_users_B = events_users_B.reset_index()
events_users_B['users_part_1event'] = (events_users_B['users'] / new_users_B * 100).round(1)
events_users_B

In [ ]:
plt.figure(figsize=(16,8))
plt.bar(events_users_A247['event'], events_users_A247['users_part_1event'])
plt.title('Доля пользователей, которые хоть раз совершали событие (группа B)')
plt.xlabel('Событие')
plt.grid()
plt.ylabel('Доля пользователей в группе');

**Вывод**

Воронка событий:

1. MainScreenAppear - Появление "Главного экрана"
2. OffersScreenAppear - Появление экрана "Предложения"
3. CartScreenAppear - Появление экрана "Корзина"
4. PaymentScreenSuccessful - Появление экрана "Платеж успешен"
5. Tutorial - Руководство пользователя

### Анализ воронки

In [ ]:
events_users

In [ ]:
fig = go.Figure(go.Funnel(x = events_users['users'], y = events_users['event'], textinfo = "value+percent initial+percent previous"))
fig.show();

Cобытие "Tutorial" непотнятно как может помочь нам с практическими результатами и какая от него польза. Будем проводить анализ, не принимая его во внимание.

In [ ]:
events_users['welcome_from_last'] = (events_users['users'] / events_users['users'].shift(1,fill_value=0) * 100).round(1)
events_users.loc[0,'welcome_from_last'] = 100
events_users['welcome_from_start'] = (events_users['users'] / events_users.loc[0,'users'] * 100).round(1)
events_users

**Промежуточный вывод**

Из первой воронки видно, многие что много пользователей (около 38%) не попадают даже на второй экран (OffersScreenAppear) с выбором товара. Возможно есть какая-то проблема на первом экране (MainScreenAppear) - надо об этом сообщить тестировщикам и обязательно выснить причину. <br>
Из этой воронки видно, что почти половина пользователей доходит до страницы с успешной оплатой (PaymentScreenSuccessful) и это очень хороший результат.

**Вывод**

Порядок событий в воронке: <br>
- MainScreenAppear - появление "Главного экрана"
- OffersScreenAppear - появление экрана "Предложения"
- CartScreenAppear - появление экрана "Корзина"
- PaymentScreenSuccessful - появление экрана "Платеж успешен"
- Tutorial - экран "Руководство пользователя" <br>

На второй этап воронки попадает 62% пользователей, 38% соответственно не доходят до второго этапа. Веб-аналитикам стоит выяснить почему так происходит, наверняка есть возможность исправить ситуацию. <br>
Событие "Tutorial" не несет полезной нагрузки, поэтому его можно не принимать во внимание. <br>
Пройдя по воронке, до экрана "PaymentScreenSuccessful" доходят 48% пользователей, то есть те кто становиться покупателями. Это очень хороший результат.

## Изучение результатов эксперимента

### Количество пользователей в каждой экспериментальной группе

In [ ]:
users_by_group = logs.groupby('group')['user'].nunique()
#users_by_group['sumA'] = users_by_group['A246'] + users_by_group['A247']
users_by_group

### A/A-тест

Для проведения тестов составим таблицу "event_group_test", где указано число пользователей в воронке, идущей по событиям и по всем группам, исключим событие "Tutorial".

In [ ]:
event_group_test = pd.DataFrame(columns = ['event', 'A246', 'A247', 'B'])
event_group_test['event'] = events_users_A246['event']
event_group_test['A246'] = events_users_A246['users']
event_group_test['A247'] = events_users_A247['users']
event_group_test['B'] = events_users_B['users']
event_group_test.drop(labels = [4],axis = 0, inplace = True)
event_group_test.loc[4] = ['AnyScreen', users_by_group['A246'], users_by_group['A247'], users_by_group['B']]
event_group_test

Напишем функцию "my_z_test" для проведения тестов, на вход подаются 3 параметра: группа1, группа2 и уровень статистической значимости.

In [ ]:
def my_z_test(s1, c1, s2, c2):
    alpha = .05 / 16 #критический уровень статистической значимости с поправкой Бонферрони
    # пропорция для первой группы
    p1 = s1 / c1
    # пропорция для второй группы
    p2 = s2 / c2
    # пропорция для комбинированной группы
    p_comb = (s1 + s2) / (c1 + c2)
    # разница пропорций
    diff = p1 - p2
    # вычисляем z-статистику
    z_value = diff / math.sqrt(p_comb * (1 - p_comb) * (1 / c1 + 1 / c2))
    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1) 
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    print('p-значение:',p_value)
    if (p_value < alpha):
        print("Отвергаем нулевую гипотезу: между долями есть значимая разница\n")
    else:
        print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными\n")

In [ ]:
for i in range(4):
    event_group_test.loc[i, "A246"]
    print('Событие:',event_group_test.loc[i, 'event'])
    my_z_test(event_group_test.loc[i, "A246"], 
              event_group_test.loc[4, "A246"], 
              event_group_test.loc[i, "A247"], 
              event_group_test.loc[4, "A247"])

**Вывод**

По результатам тестирования для всех событий разница не оказалось значимой, поэтому эти группы считаем контрольными.

### A/B-тесты

In [ ]:
for i in range(4):
    event_group_test.loc[i, "A246"]
    print('Событие:',event_group_test.loc[i, 'event'])
    my_z_test(event_group_test.loc[i, "A246"], 
              event_group_test.loc[4, "A246"], 
              event_group_test.loc[i, "B"], 
              event_group_test.loc[4, "B"])

In [ ]:
for i in range(4):
    event_group_test.loc[i, "A247"]
    print('Событие:',event_group_test.loc[i, 'event'])
    my_z_test(event_group_test.loc[i, "A247"], 
              event_group_test.loc[4, "A247"], 
              event_group_test.loc[i, "B"], 
              event_group_test.loc[4, "B"])

In [ ]:
for i in range(4):
    event_group_test.loc[i, "A246"]
    print('Событие:',event_group_test.loc[i, 'event'])
    my_z_test(event_group_test.loc[i, "A246"] + event_group_test.loc[i, "A247"], 
              event_group_test.loc[4, "A246"] + event_group_test.loc[4, "A247"], 
              event_group_test.loc[i, "B"], 
              event_group_test.loc[4, "B"])

**Вывод**

Значимой разницы между группами во всех тестах не выявлено.

## Выводы

В ходе проверки данных выяснилось, что данные предоставлены с 25 июля по 7 августа. Данные в июле характеризуются малой событийностью. Скорее всего это было какое-то тестирование, а сбор данных начался уже со второй недели. Данные за июль были исключены. Оставили период с 2019-07-31 21:00:00. Потеряли 0.82% событий, что существенно не должно повлиять на дальнейший анализ. <br>

Среднее количествово событий пользователя равно 32, но "длинный хвост" указывает на наличие "выбросов" в исходных данных, поэтому рациональней использовать медиану равную 20. <br>
Количество пользователей в каждой группе осталось примерно 2500, а событий около 80000 на каждую группу. Достаточно ровные данные.

Определен порядок событий в воронке:
- MainScreenAppear - появление "Главного экрана" (117889 событий)
- OffersScreenAppear - появление экрана "Предложения" (46531 событий)
- CartScreenAppear - появление экрана "Корзина" (42343 событий)
- PaymentScreenSuccessful - появление экрана "Платеж успешен" (33951 событий)
- Tutorial - экран "Руководство пользователя" (1010 событий)<br>
Число пользователей совершивших самое популярное событие "MainScreenAppear" - 7423, примерно по 2500 на группу. <br>

На второй этап воронки событий попадает 62% пользователей, 38% соответственно не доходят до второго этапа. Веб-аналитикам стоит выяснить почему так происходит, наверняка есть возможность исправить ситуацию. <br>
От события "OffersScreenAppear" до события "CartScreenAppear" не доходит 19%. Возможно удержать пользователя помогли бы маркетинговые уловки в виде внезапных скидок или подарков. <br>
От появления экрана "Корзины" до покупки не доходит 5%. Могут быть проблемы с оплатой. Стоит максимально упростить эту процедуру и (или) добавить иные способы оплаты. <br>
Событие "Tutorial" не несет полезной нагрузки, поэтому его можно не принимать во внимание. <br>
До экрана "PaymentScreenSuccessful" доходят 48% от первоначального количества пользователей, то есть те кто становиться покупателями. Это очень хороший результат. <br>

В ходе тестирования были проведены 16 экспериментов (уровень статистической значимости - 5/16%):
- A246/A247 - 4 эксперемента (для каждого события)
- A246/B - 4 эксперемента (для каждого события)
- A247/B - 4 эксперемента (для каждого события)
- A246+A247/B - 4 эксперемента (для каждого события) <br>

По результатам тестирования групп A246 и A247 для всех событий разница не оказалось значимой, поэтому эти группы считаем контрольными. <br>

В результате каждого A/A/B-теста значимой разницы между группами не выявлено. Поэтому можно утверждать, что на поведение пользователей изменение шрифта значимого эффекта не оказало. Тестирование можно назвать успешным - изменение шрифта не повлияло на поведение пользователей.